In [19]:
import pandas as pd

In [20]:
"""Preparing the data"""

# read the csv
schooldirectory = pd.read_csv("data/2016_DOE_High_School_Directory.csv")
schoolsat = pd.read_csv("data/2012_SAT_Results.csv")
schoolregent = pd.read_csv("data/2014_-_2017_Regents_modified.csv")
schoolperformance = pd.read_csv("data/2016_DOE_High_School_Performance__Directory.csv")

# Modify datasets before joining
schoolsat['Num of SAT Test Takers'] = pd.to_numeric(schoolsat['Num of SAT Test Takers'], errors='coerce')
schoolsat = schoolsat.dropna(subset=['Num of SAT Test Takers'])

schoolsat['SAT Critical Reading Avg. Score'] = pd.to_numeric(schoolsat['SAT Critical Reading Avg. Score'], errors='coerce')
schoolsat['SAT Math Avg. Score'] = pd.to_numeric(schoolsat['SAT Math Avg. Score'], errors='coerce')
schoolsat['SAT Writing Avg. Score'] = pd.to_numeric(schoolsat['SAT Writing Avg. Score'], errors='coerce')

# join data
schoolperformance = schoolperformance.join(schooldirectory.set_index('dbn'), on='dbn')
schoolperformance = schoolperformance.join(schoolsat.set_index('dbn'), on='dbn')
schoolperformance = schoolperformance.join(schoolregent.set_index('dbn'), on='dbn')
schoolperformance = schoolperformance.fillna(0)

In [21]:
def method_1():
    """
    Method 1
    Algorithm to rank the schools based on brute force. Better the stats better the ranking.

    Personal opinion of accuracy based on the data shown - (8/10)
    *My personal opinion is based of the "rep" of the school.
    """ 
    # Basic Stats
    basic_stats = (
        schoolperformance["graduation_rate_2013"] + schoolperformance["graduation_rate_2014"] + schoolperformance["college_career_rate_2013"] +          schoolperformance["college_career_rate_2014"] + schoolperformance["pct_stu_enough_variety_2014"] + schoolperformance["pct_stu_safe_2014"]
    )

    # SAT
    sat_stats = (
        schoolperformance["SAT Critical Reading Avg. Score"].astype(int) + schoolperformance["SAT Math Avg. Score"].astype(int) +                        schoolperformance["SAT Writing Avg. Score"].astype(int)
    )

    #Regent
    regent_stats = (
        schoolperformance["Mean Score"]
    )

    schoolperformance["score"] = basic_stats + sat_stats + regent_stats

    return schoolperformance["score"]

In [22]:
def method_2():
    """
    Method 2
    Algorithm to rank the schools by average and personal opinion. This way the score doesn't solely rely on the SAT scores of students instead      it is split in a ratio. In this case the ratio is 33:33:33. The score can be considered as a numeric grade for each schools performance.

    Personal opinion of accuracy based of the data shown - (9/10)
    *My personal opinion is based of the "rep" of the school.
    """

    # Basic Stats
    basic_stats = (
            (schoolperformance["graduation_rate_2013"] + schoolperformance["graduation_rate_2014"] + schoolperformance["college_career_rate_2013"] + schoolperformance["college_career_rate_2014"] + schoolperformance["pct_stu_enough_variety_2014"] + schoolperformance["pct_stu_safe_2014"])/600
        )

    # SAT 
    sat_stats = (
        (schoolperformance["SAT Critical Reading Avg. Score"].astype(int) + schoolperformance["SAT Math Avg. Score"].astype(int) + schoolperformance["SAT Writing Avg. Score"].astype(int)) /2400
        )
    
    #Regent
    regent_stats = (
        schoolperformance["Mean Score"]/100
    )

    schoolperformance["score"] = (basic_stats + sat_stats + regent_stats)/3

    return schoolperformance["score"]

In [23]:
"""Method 1"""
method_1()

# sorts the school by the score 
schoolperformance.sort_values(by=['score'], ascending=False, inplace=True)
# gets rid of the ones without data
schoolperformance_display = schoolperformance[["dbn", "school_name", "borough", "total_students", "score"]]
best_schools = schoolperformance_display.head(10)
best_schools

,dbn,school_name,borough,total_students,score
46,02M475,Stuyvesant High School,Manhattan,3296,2767.566667
186,10X445,Bronx High School of Science,Bronx,3015,2637.556667
430,31R605,Staten Island Technical High School,Staten Island,1247,2632.031034
363,25Q525,Townsend Harris High School,Queens,1132,2585.806250
192,10X696,High School of American Studies at Lehman College,Bronx,376,2574.656250
5,01M696,Bard High School Early College,Manhattan,545,2512.313333
95,05M692,"High School for Mathematics, Science and Engin...",Manhattan,470,2485.268966
396,28Q687,Queens High School for the Sciences at York Co...,Queens,426,2479.360714
231,13K430,Brooklyn Technical High School,Brooklyn,5447,2463.847059
33,02M416,Eleanor Roosevelt High School,Manhattan,541,2421.433333


In [24]:
"""Method 2"""
method_2()

# sorts the school by the score 
schoolperformance.sort_values(by=['score'], ascending=False, inplace=True)
# gets rid of the ones without data
schoolperformance_display = schoolperformance[["school_name", "borough", "total_students", "score"]]
best_schools = schoolperformance_display.head(10)
best_schools

,school_name,borough,total_students,score
46,Stuyvesant High School,Manhattan,3296,0.918556
186,Bronx High School of Science,Bronx,3015,0.896439
430,Staten Island Technical High School,Staten Island,1247,0.895798
363,Townsend Harris High School,Queens,1132,0.890188
192,High School of American Studies at Lehman College,Bronx,376,0.876632
5,Bard High School Early College,Manhattan,545,0.853822
95,"High School for Mathematics, Science and Engin...",Manhattan,470,0.853535
33,Eleanor Roosevelt High School,Manhattan,541,0.850056
396,Queens High School for the Sciences at York Co...,Queens,426,0.844536
231,Brooklyn Technical High School,Brooklyn,5447,0.843518
